In [3]:
!pip install sagemaker --quiet --upgrade --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
conda 22.9.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
docker-compose 1.29.2 requires jsonschema<4,>=2.5.1, but you have jsonschema 4.17.3 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.111 requires botocore==1

In [51]:
import json
import sagemaker
import boto3
import os
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [58]:
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # ml.g5.2xlarge
#os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # ml.g5.48xlarge

In [60]:
region = boto3.Session().region_name
region

'us-west-2'

In [96]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'tiiuae/falcon-7b-instruct', 
    #'HF_MODEL_ID' : 'huggingface-llm-falcon-7b-instruct-bf16',
	'SM_NUM_GPUS': json.dumps(1)
}

endpoint_name = 'sm-contract-audit'

In [97]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

In [98]:
# from sagemaker.serverless import ServerlessInferenceConfig

# # Specify MemorySizeInMB and MaxConcurrency in the serverless config object
# serverless_config = ServerlessInferenceConfig(
#     memory_size_in_mb=3072, max_concurrency=10, #3072, 4096, 5120, or 6144 . 
# )

# # deploy the endpoint endpoint
# predictor = huggingface_model.deploy(
#     serverless_inference_config=serverless_config,
#     endpoint_name=endpoint_name,
# )

In [99]:
# help(HuggingFaceModel)

In [100]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",  
	container_startup_health_check_timeout=300,
    endpoint_name=endpoint_name
  )

--------------!

### Endpoint Inference

In [159]:
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to  ",
})

[{'generated_text': 'My name is Julien and I like to "play" with my camera.\nI\'m a French photographer based in Paris. I\''}]

In [163]:
predictor.predict({
	"inputs": """Generate response for the below text, without repeating the Text in the response
            
            Text:
            ```My name is Julien and I like to ```
            
            Response:
            """
})

[{'generated_text': 'Generate response for the below text, without repeating the Text in the response\n            \n            Text:\n            ```My name is Julien and I like to ```\n            \n            Response:\n            ```My name is Julien and I like to code.```'}]

In [ ]:
# ml.c5.2xlarge
# https://us-east-1.console.aws.amazon.com/servicequotas/home/services/sagemaker/quotas

In [67]:
# deploy model to SageMaker Inference
# predictor = huggingface_model.deploy(
# 	initial_instance_count=1,
# 	instance_type="ml.c5.2xlarge",  
# 	container_startup_health_check_timeout=300,
#   endpoint_name=endpoint_name
#   )

In [85]:
def query_endpoint(payload):
    """Query endpoint and print the response"""
    response = predictor.predict(payload)
    #print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

In [73]:
# payload = {
#     "inputs": """"User: Can you identify the security issues in the following smart contract code within ```?

#                 Code:
#                 ```
#                 // SPDX-License-Identifier: MIT
#                 pragma solidity ^0.8.17;

#                 contract CrowdFunding {
#                     struct Campaign {
#                         address owner;
#                         string title;
#                         string description;
#                         uint256 target;
#                         uint256 deadline;
#                         uint256 amountCollected;
#                         string image;
#                         address[] donators;
#                         uint256[] donations;
#                     }

#                     mapping(uint256 => Campaign) public campaigns;

#                     uint256 public numberOfCampaigns = 0;

#                     function createCampaign(
#                         address _owner,
#                         string memory _title,
#                         string memory _description,
#                         uint256 _target,
#                         uint256 _deadline,
#                         string memory _image
#                     ) public returns (uint256) {
#                         Campaign storage campaign = campaigns[numberOfCampaigns];

#                         require(
#                             campaign.deadline < block.timestamp,
#                             "The deadline should be a date in the future."
#                         );

#                         campaign.owner = _owner;
#                         campaign.title = _title;
#                         campaign.description = _description;
#                         campaign.target = _target;
#                         campaign.deadline = _deadline;
#                         campaign.amountCollected = 0;
#                         campaign.image = _image;

#                         numberOfCampaigns++;

#                         return numberOfCampaigns - 1;
#                     }

#                     function donateToCampaign(uint256 _id) public payable {
#                         uint256 amount = msg.value;

#                         Campaign storage campaign = campaigns[_id];

#                         campaign.donators.push(msg.sender);
#                         campaign.donations.push(amount);

#                         campaign.amountCollected = campaign.amountCollected + amount;
#                     }

#                     function withdraw(uint256 _id) public {
#                         Campaign storage campaign = campaigns[_id];

#                         require(
#                             campaign.owner == msg.sender,
#                             "Only the owner of the campaign can withdraw the funds."
#                         );

#                         require(
#                             campaign.amountCollected >= campaign.target,
#                             "The campaign has not reached its target yet."
#                         );

#                         require(
#                             campaign.deadline < block.timestamp,
#                             "The deadline has not passed yet."
#                         );

#                         (bool sent, ) = payable(campaign.owner).call{
#                             value: campaign.amountCollected
#                         }("");

#                         if (sent) {
#                             campaign.amountCollected = 0;
#                         }
#                     }

#                     function getCampaign(uint256 id)
#                         public
#                         view
#                         returns (
#                             address,
#                             string memory,
#                             string memory,
#                             uint256,
#                             uint256,
#                             uint256,
#                             string memory
#                         )
#                     {
#                         Campaign memory campaign = campaigns[id];
#                         return (
#                             campaign.owner,
#                             campaign.title,
#                             campaign.description,
#                             campaign.target,
#                             campaign.deadline,
#                             campaign.amountCollected,
#                             campaign.image
#                         );
#                     }
#                 }
#                 ```

#                 Assistant: Sure! Let me analyze the smart contract code for security issues.
#                 List the security issues in the above code ?
#                 """,
#     "parameters": {
#         "do_sample": True,
#         "top_p": 0.9,
#         "temperature": 0.9,
#         # "max_new_tokens":1024
#     }
# }
# predictor.predict(payload)

In [178]:
payload = {
    "inputs": """ Analyse the below solidity smart contract code:

                ```
                // SPDX-License-Identifier: MIT
                pragma solidity ^0.8.17;

                contract CrowdFunding {
                    struct Campaign {
                        address owner;
                        string title;
                        string description;
                        uint256 target;
                        uint256 deadline;
                        uint256 amountCollected;
                        string image;
                        address[] donators;
                        uint256[] donations;
                    }

                    mapping(uint256 => Campaign) public campaigns;

                    uint256 public numberOfCampaigns = 0;

                    function createCampaign(
                        address _owner,
                        string memory _title,
                        string memory _description,
                        uint256 _target,
                        uint256 _deadline,
                        string memory _image
                    ) public returns (uint256) {
                        Campaign storage campaign = campaigns[numberOfCampaigns];

                        require(
                            campaign.deadline < block.timestamp,
                            "The deadline should be a date in the future."
                        );

                        campaign.owner = _owner;
                        campaign.title = _title;
                        campaign.description = _description;
                        campaign.target = _target;
                        campaign.deadline = _deadline;
                        campaign.amountCollected = 0;
                        campaign.image = _image;

                        numberOfCampaigns++;

                        return numberOfCampaigns - 1;
                    }

                    function donateToCampaign(uint256 _id) public payable {
                        uint256 amount = msg.value;

                        Campaign storage campaign = campaigns[_id];

                        campaign.donators.push(msg.sender);
                        campaign.donations.push(amount);

                        campaign.amountCollected = campaign.amountCollected + amount;
                    }

                    function withdraw(uint256 _id) public {
                        Campaign storage campaign = campaigns[_id];

                        require(
                            campaign.owner == msg.sender,
                            "Only the owner of the campaign can withdraw the funds."
                        );

                        require(
                            campaign.amountCollected >= campaign.target,
                            "The campaign has not reached its target yet."
                        );

                        require(
                            campaign.deadline < block.timestamp,
                            "The deadline has not passed yet."
                        );

                        (bool sent, ) = payable(campaign.owner).call{
                            value: campaign.amountCollected
                        }("");

                        if (sent) {
                            campaign.amountCollected = 0;
                        }
                    }

                    function getCampaign(uint256 id)
                        public
                        view
                        returns (
                            address,
                            string memory,
                            string memory,
                            uint256,
                            uint256,
                            uint256,
                            string memory
                        )
                    {
                        Campaign memory campaign = campaigns[id];
                        return (
                            campaign.owner,
                            campaign.title,
                            campaign.description,
                            campaign.target,
                            campaign.deadline,
                            campaign.amountCollected,
                            campaign.image
                        );
                    }
                }
                ```

                List the security issues in the above code ?
                
                
                """,
    "parameters": {
        #"do_sample": True,
        "top_p": 0.9,
        #"temperature": 0.9,
        "stop": ["<|endoftext|>", "</s>"]
        #"max_new_tokens":1024
    }
}
# predictor.predict(payload)

In [179]:
query_endpoint(payload)

 Output:  Analyse the below solidity smart contract code:

                ```
                // SPDX-License-Identifier: MIT
                pragma solidity ^0.8.17;

                contract CrowdFunding {
                    struct Campaign {
                        address owner;
                        string title;
                        string description;
                        uint256 target;
                        uint256 deadline;
                        uint256 amountCollected;
                        string image;
                        address[] donators;
                        uint256[] donations;
                    }

                    mapping(uint256 => Campaign) public campaigns;

                    uint256 public numberOfCampaigns = 0;

                    function createCampaign(
                        address _owner,
                        string memory _title,
                        string memory _description,
                        uint256 _target,


## Delete Endpoint

In [180]:
predictor.delete_model()
predictor.delete_endpoint()